In [1]:
import pandas as pd 
import numpy as np
import datetime as dt
import os
from sklearn import linear_model
import warnings

In [2]:
warnings.filterwarnings('ignore')
PATH = "D:/csmar解压数据/"  # 从csmar下载的原始数据的存放路径
FILETYPE = "csv"  # 原始数据的文件格式
DATE = ["Trddt"]  # 日期索引，根据raw data列名取值，另有取值'Accper'
CODE = ["Stkcd"]  # 代码索引，根据raw data列名取值，另有取值'Symbol'

# 1. Data Extraction
定义用来从存放数据的文件夹中提取所需数据的函数。原始数据格式为表头表格，而目标数据格式一般是pivot table，准确地说，是横轴为样本、纵轴为日期、内容为取值的二维表格，并且在函数中会把日期转换为pandas.DatetimeIndex类型

In [3]:
# 从原始数据中提取出数据透视表，并且会把index转化为pandas的日期索引
def extract_variable(path:str="", fileType=FILETYPE, variable:str="", index:list=DATE, columns:list=CODE):
    """
    path: fold address
    fileType: 
    variable: column name of the target value in raw data
    index: column name of the target index in raw data
    columns: column name of the target columns in raw data
    """
    # collect all data needed in the fold and print their names
    filePathList = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if name.endswith("."+fileType):
                filePath = os.path.join(root, name)
                filePathList.append(filePath)
                print(filePath)
    
    # read files into DataFrame one by one
    dfList = []
    for filePath in filePathList:
        df = eval("pd.read_"+fileType+"(filePath)")
        dfList.append(df)
      
    # concatenate DFs into one large table
    all_df = pd.concat(dfList)
    
    # 只选沪深A股，若原始数据中有"Markettype"列
    flag = False
    if "Markettype" in all_df.columns:
        all_df = all_df[all_df.loc[:,"Markettype"].isin([1,4])]  # 1，4在csmar中代表沪深主板
        flag = True
    
    pivot_df = pd.pivot_table(all_df, values=variable, index=index, columns=columns) 
    
    # 只选沪深A股，若若原始数据中无"Markettype"列，只能靠股票代码的数据(是int)来区分。不知两种方法快慢
    if flag == False:
        df1 = pivot_df.loc[:, 0:9999]
        df2 = pivot_df.loc[:, 600000:609999]
        pivot_df = pd.concat([df1, df2], axis=1)
    
    # 转换日期类型
    pivot_df.index = pd.DatetimeIndex(pivot_df.index)
    
    return pivot_df

In [4]:
def extract_variable_oneFile(filePath:str="", fileType=FILETYPE, variable:str="", index:list=DATE, columns:list=CODE):
    df = eval("pd.read_"+fileType+"(filePath)")
    
    # 只选沪深A股
    flag = False
    if "Markettype" in df.columns:
        df = df[df.loc[:,"Markettype"].isin([1,4])]
        flag = True
    
    pivot_df = pd.pivot_table(df, values=variable, index=index, columns=columns) 
    
    # 只选沪深A股
    if flag == False:
        df1 = pivot_df.loc[:, 0:9999]
        df2 = pivot_df.loc[:, 600000:609999]
        pivot_df = pd.concat([df1, df2], axis=1)
               
    pivot_df.index = pd.DatetimeIndex(pivot_df.index)
    return pivot_df

# 2. Factor Construction

## 1. std_dolvol
Monthly standard deviation of daily RMB trading volume

In [ ]:
def get_std_dolvol(valvol:pd.DataFrame, path:str):
    std_dolvol = valvol.resample("M").apply(np.std)
    
    std_dolvol.T.to_csv(path)
    # return std_dolvol.T

In [ ]:
df = extract_variable(path=PATH+"01日个股回报率", variable="Dnvaltrd").loc['2000-01-01':'2021-11-30',:]
df

In [ ]:
get_std_dolvol(df, "C:/Users/ShaunMarx/助研_人大/Ourproject01_China_stock/Data/factors/std_dolvol.csv")

## 2. zerotrade
Turnover weighted number of zero trading days in month t − 1.  
https://www.sciencedirect.com/science/article/abs/pii/S0304405X0600064X?via%3Dihub  
己见：也许是容易引起误会的名称，实质上重心在turnover的倒数上，加入zerotradeday是针对停牌的修正。数值越大，流动性越低

In [ ]:
def get_zerotrade(volume:pd.DataFrame, turnover:pd.DataFrame, path:str):
    
    notd = volume.iloc[:,0].resample("M").apply(len)
    noztd = volume.resample("M").apply(pd.isna).resample("M").apply(sum)
    sot = turnover.resample("M").sum().replace(0, np.nan)
    #df = ( noztd + 1 / sot ) / notd
    df = noztd + 1 / sot
    df = df.apply(lambda x: x / notd)
    df.T.to_csv(path)
    
    #return df.T

In [ ]:
df2 = extract_variable(path=PATH+"03个股日交易衍生指标",variable="Turnover",
                       index=['TradingDate'],columns=['Symbol']).loc['2000-01-01':'2021-11-30',:]
df2

In [ ]:
get_zerotrade(volume=df, turnover=df2, path='C:/Users/ShaunMarx/助研_人大/Ourproject01_China_stock/Data/factors/zerotrade.csv')

## 3. atr
The abnormal turnover ratio

In [ ]:
mkt_tr = pd.read_excel('D:/csmar解压数据/A股市场换手率.xlsx', index_col="日期", date_parser="%Y-%m-%d")
mkt_tr

In [ ]:
stk_tr = extract_variable(path=PATH+"03个股日交易衍生指标",variable="Turnover",
                       index=['TradingDate'],columns=['Symbol']).loc['1999-12-01':'2021-11-30',:]
stk_tr

In [ ]:
def get_atr(stk_tr:pd.DataFrame, mkt_tr:pd.Series, path:str):
    res_df = pd.DataFrame()
    m_train_start = pd.date_range(start="1999-12-01", end="2021-05-01", freq="MS") # MS: month start frequency
    m_train_end = pd.date_range(start="2000-05-31", end="2021-10-31", freq="M")  # M: month end frequency
    m_test_start = pd.date_range(start="2000-06-01", end="2021-11-01", freq="MS")
    m_test_end = pd.date_range(start="2000-06-30", end="2021-11-30", freq="M")
    m_n = range(len(m_train_start))
    for i in m_n:
        stk_tr_train = stk_tr.loc[m_train_start[i]:m_train_end[i],:]
        stk_tr_test = stk_tr.loc[m_test_start[i]:m_test_end[i],:]
        mkt_tr_train = mkt_tr.loc[m_train_start[i]:m_train_end[i]]
        mkt_tr_test = mkt_tr.loc[m_test_start[i]:m_test_end[i]]
        for stk in stk_tr.columns:
            y_train = stk_tr_train[stk]
            y_test = stk_tr_test[stk]
            
            # x即市场换手率必定无缺，但y即个股换手率可能有缺（如停牌），故x要随y丢弃相应行
            y_train.dropna(inplace=True)
            x_train = mkt_tr_train.loc[y_train.index]
            y_test.dropna(inplace=True)
            x_test = mkt_tr_test.loc[y_test.index]
            
            if (y_train.empty or y_test.empty):
                continue
                
            regr = linear_model.LinearRegression()
                #x_y_train = pd.concat([x_train, y_train], axis=1)
                #x_y_train.dropna(inplace=True)
                #x_y_train.columns = ['x', 'y']
                
            regr.fit(x_train.to_numpy().reshape(-1,1), y_train.to_numpy())
            y_pred = regr.predict(x_test.to_numpy().reshape(-1,1))
            d_atr = y_test.to_numpy() - y_pred
            m_atr = d_atr.sum()
            res_df.loc[m_test_end[i],stk] = m_atr
    
    res_df.T.to_csv(path)
    return res_df.T

In [ ]:
get_atr(stk_tr=stk_tr, mkt_tr=mkt_tr, path='C:/Users/ShaunMarx/助研_人大/Ourproject01_China_stock/Data/factors/atr.csv')

 ## 4. std_turn
 Monthly standard deviation of daily share turnover

In [ ]:
def get_std_turn(turnover:pd.DataFrame, path:str):
    std_turn = turnover.resample("M").apply(np.std)
    
    std_turn.T.to_csv(path)
    # return std_turn.T

In [ ]:
to = extract_variable(path=PATH+"03个股日交易衍生指标",variable="Turnover",
                       index=['TradingDate'],columns=['Symbol']).loc['2000-01-01':'2021-11-30',:]
to

In [ ]:
get_std_turn(to, "C:/Users/ShaunMarx/助研_人大/Ourproject01_China_stock/Data/factors/std_turn.csv")

## 5. ill
Average of daily (absolute return/RMB volume) in month t

In [ ]:
abs_ret = extract_variable(path=PATH+"01日个股回报率", variable="Dretwd").loc['2000-01-01':'2021-11-30',:]
abs_ret

In [ ]:
val_trd = extract_variable(path=PATH+"01日个股回报率", variable="Dnvaltrd").loc['2000-01-01':'2021-11-30',:]
val_trd

In [ ]:
def get_ill(abs_ret:pd.DataFrame, val_trd:pd.DataFrame, path:str):
    abs_ret = abs_ret * 1e8
    res  = abs_ret / val_trd
    res = res.resample("M").mean()
    
    res.T.to_csv(path)
    return res

In [ ]:
get_ill(abs_ret=abs_ret,val_trd=val_trd,path="C:/Users/ShaunMarx/助研_人大/Ourproject01_China_stock/Data/factors/ill.csv")

## 5. mve
Natural log of market capitalization at end of month t-1

In [ ]:
mktval = extract_variable(path=PATH+"01日个股回报率", variable="Dsmvtll").loc['2000-01-01':'2021-11-30',:]
mktval

In [ ]:
def get_mve(mktval:pd.DataFrame, path:str):
    res = mktval.resample('M').last()
    res = res.applymap(lambda x: np.log(x))
    
    res.T.to_csv(path)
    return res

In [ ]:
get_mve(mktval=mktval,path="C:/Users/ShaunMarx/助研_人大/Ourproject01_China_stock/Data/factors/mve.csv")

## 6. nincr
Number of consecutive quarters (up to eight quarters) with an increase in earnings

In [ ]:
ern = extract_variable_oneFile(filePath=PATH+"04财务指标分析/FI_T5.csv",index=["Accper"],columns=["Stkcd"],
                               variable="F050701B").loc['2000-01-01':'2021-11-30',:]
ern

In [ ]:
def consec_count(s):
    i = 0
    for x in s[::-1]:
        if x == True:
            i = i+1
        else:
            break
    return i

def get_nincr(ern:pd.DataFrame, path:str):
    ern = ern.applymap(lambda x: True if x>0 else False)
    res = ern.rolling(window=8).apply(consec_count)
    
    res.T.to_csv(path)
    return res

In [ ]:
get_nincr(ern=ern,path="C:/Users/ShaunMarx/助研_人大/Ourproject01_China_stock/Data/factors/nincr.csv")

## 2.7. bmr_ia
industry-adjusted book-to-market ratio

In [ ]:
# input 1: book-to-market ratio
bmr = extract_variable_oneFile(filePath=PATH+"04财务指标分析/FI_T10.csv",
                               variable="F101001A", index=['Accper']).loc['2000-01-01':'2021-11-30',:]
bmr

In [ ]:
# input 2: stock classification
cls = pd.read_csv('D:/csmar解压数据/05上市公司行业分类表/STK_INDUSTRYCLASS.csv', index_col=[4], parse_dates=True)
df1 = cls[cls['Symbol'] >= 0]
df1 = df1[df1['Symbol'] <= 9999]
df2 = cls[cls['Symbol'] >= 600000]
df2 = df2[df2['Symbol'] <= 609999]
cls = pd.concat([df1, df2])
cls

In [ ]:
# 查看不同分类标准对股票的覆盖面，希望找到最全的，'P02**'在csmar中是分类标准代码
for i in ['P0201','P0202','P0203','P0204','P0205','P0206','P0207','P0208','P0209','P0210','P0211','P0212','P0213','P0214','P0215']:
    df = cls[cls['IndustryClassificationID'] == i]
    print(i,df['Symbol'].nunique())

In [ ]:
# 发现证监会2012版覆盖面最全
cls = cls[cls['IndustryClassificationID'] == 'P0207']
cls = cls[['Symbol','IndustryCode']]
cls

#### 专门定义一个函数用来做等权重均值作差的行业调整，适用于全部以_ia结尾的因子

In [ ]:
def EWA_industry_adjust(value_df:pd.DataFrame, industry_df:pd.DataFrame):
    """
    equally-weighted average industry-adjusted:
    x_ia_it = x_it − x_Iit
    
    params:
    value_df: a 2-dimension(rows are dates,cols are codes) table of target values
    industry_df: classification informantion, a 3-column table, cols are implement date, stock code, industry classification
    """
    industry_df.sort_index(inplace=True)
    for d in value_df.index:
        value_sec = value_df.loc[d]
        symbols = value_sec.index.sort_values()
        group = []
        for s in symbols:
            c = industry_df[industry_df.iloc[:,0] == s]
            c = c[c.index <= d]
            c = c.tail(1)
            if c.empty:
                c = np.nan
            else:
                c = c.iloc[0,1]
            group.append(c)
        means = value_sec.groupby(group).mean()
        for s in symbols:
            c = industry_df[industry_df.iloc[:,0] == s]
            c = c[c.index <= d]
            c = c.tail(1)
            if c.empty:
                value_df.loc[d,s] = np.nan
            else:
                c = c.iloc[0,1]
                m = means[c]
                value_df.loc[d,s] -= m
        print(d)
    return value_df

In [ ]:
data = EWA_industry_adjust(value_df=bmr, industry_df=cls)

In [ ]:
data = bmr_ia
data

In [ ]:
# 要删
bmr_ia = data
bmr_ia

In [ ]:
# 先做一次变频fill
bmr_ia = bmr_ia.asfreq('d')
bmr_ia.fillna(method='ffill', inplace=True)
bmr_ia

In [ ]:
index = pd.read_csv('A股市场交易日.csv', index_col=[1], parse_dates=True)
index = index.loc['1999-12-01':'2021-11-30']
index

In [ ]:
bmr_ia = bmr_ia.reindex(index.index)

In [ ]:
bmr_ia.fillna(method='ffill', inplace=True)

In [ ]:
bmr_ia

In [ ]:
bmr_ia.T.to_csv("C:/Users/ShaunMarx/助研_人大/Ourproject01_China_stock/Data/factors/bmr_ia.csv")